In [16]:
x = {184737297734959104 : {'Name': ['adam'],
                        'avaliability': {'2023-06-01': 'yes',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-06-01': True,
                                        '2023-05-25': True},
                        'goal': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'motm-vote': {},
                        'motm': {}  
                            },

     462409072176791564 : {'Name': ['matty'],
                        'avaliability': {'2023-06-01': 'yes',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-06-01': True,
                                        '2023-05-25': True},
                        'goal': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 1},
                        'motm-vote': {},
                        'motm': {}                             
                           },

     265573788232581120 : {'Name': ['tommy'],
                        'avaliability': {'2023-06-01': 'yes',
                                        '2023-05-25': 'no'},
                        'paid': {'2023-06-01': True},
                        'goal': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'motm-vote': {},
                        'motm': {}       
                            },   

     454252561063477250 : {'Name': ['noz', 'lawrence'],
                        'avaliability': {'2023-06-01': 'yes',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-06-01': True,
                                        '2023-05-25': True},
                        'goal': {'2023-06-01': 0,
                                '2023-05-25': 1},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 0}, 
                        'motm-vote': {},
                        'motm': {}      
                            },  

     462410416048308225 : {'Name': ['tom', 'Tom b'],
                        'avaliability': {'2023-06-01': 'no',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-05-25': True},
                        'goal': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'motm-vote': {},
                        'motm': {}       
                            },

     462654687225708555 : {'Name': ['tk', 'thomas'],
                        'avaliability': {'2023-06-01': 'yes',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-06-01': True,
                                        '2023-05-25': True},
                        'goal': {'2023-06-01': 1,
                                '2023-05-25': 1},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'motm-vote': {},
                        'motm': {}     
                            },

     265576326788808704 : {'Name': ['jack'],
                        'avaliability': {'2023-06-01': 'yes',
                                        '2023-05-25': 'yes'},
                        'paid': {'2023-06-01': True,
                                        '2023-05-25': True},
                        'goal': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'assist': {'2023-06-01': 0,
                                '2023-05-25': 0},
                        'motm-vote': {},
                        'motm': {}       
                            }

     }

In [1]:
import json

# read data from json
path = '//TRUENAS/Misc_storage/5aside_discord_bot/user_data/team_data.json'

with open(path, 'r') as f:
    data = json.load(f)

# save data to json
# with open(path, 'w') as f:
#     json.dump(x, f, indent=4)




In [12]:
# autoreload modules
%load_ext autoreload
%autoreload 2

from player import Player

team = {}

for id, val in data.items():

    user = Player(name = val['Name'], 
                id = id,
                availability = val['avaliability'],
                paid = val['paid'],
                goal = val['goal'],
                assist = val['assist'],
                motm_vote = val['motm-vote'],
                motm = val['motm'],
                )
    
    team[user.name[0]] = user

print(team.keys())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
dict_keys(['adam', 'matty', 'tommy', 'noz', 'tom', 'tk', 'jack'])


In [15]:
# extract data from player class to json file
data_out = {}
for name, val in team.items():
    data_out[val.id] = {}
    data_out[val.id]['Name'] = val.name
    data_out[val.id]['avaliability'] = val.availability
    data_out[val.id]['paid'] = val.paid
    data_out[val.id]['goal'] = val.goal
    data_out[val.id]['assist'] = val.assist
    data_out[val.id]['motm-vote'] = val.motm_vote
    data_out[val.id]['motm'] = val.motm

In [16]:

data == data_out


True

In [18]:
data_out['184737297734959104']

{'Name': ['adam'],
 'avaliability': {'2023-06-01': 'yes', '2023-05-25': 'yes'},
 'paid': {'2023-06-01': True, '2023-05-25': True},
 'goal': {'2023-06-01': 0, '2023-05-25': 0},
 'assist': {'2023-06-01': 0, '2023-05-25': 0},
 'motm-vote': {},
 'motm': {}}